# GPU Version to run on MARCC

This notebook is assummed to be launched from the top level of the AutomaticLP github repo (where the notebook file is)

This also assummes that you've already cloed the tensorflow object detection project
https://github.com/tensorflow/models (since you need to do this to add the python path to  the bashrc file which has to be done before launching jupyter lab

Run with the LPproject kernel

## Setup / Installations

Add
<code>
export PYTHONPATH=$PYTHONPATH:`pwd`
export PYTHONPATH=$PYTHONPATH:`pwd`/slim

to your ~/.bashrc file where pwd is the full path to the models/research directory

### Tests that Tensorflow GPU is installed & this notebook can see the GPU 

These don't need to be run everytime but are here as a sanity / debugging check

In [ ]:
# test that tensorflow can access a GPU
import tensorflow as tf
tf.test.is_built_with_cuda()

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
print ('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


In [ ]:
%%bash 
cd models/research
FILE=protobuf.zip
if [ ! -f "$FILE" ]; then
    wget -O protobuf.zip https://github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip
    unzip protobuf.zip
fi
./bin/protoc object_detection/protos/*.proto --python_out=.
pip install --user .

In [ ]:
!ml cuda/9.0
!export CUDA_VISIBLE_DEVICES=1
!echo ${CUDA_VISIBLE_DEVICES}

## Generating Datasets

The ScriptForBashFile generates both the training and test data and TFRecords from the image files

(For now hard coded to 100 images - TODO: switch back to user input, note then this won't be able to run with a bash script)

In [ ]:
%run ScriptForBashFile

## Train


In [ ]:
# If training output directory dosen't exist create it
!mkdir -p trainingOutput

In [1]:
# This makes the output of the next cell write to the file
# https://stackoverflow.com/questions/45200375/stdout-redirect-from-jupyter-notebook-is-landing-in-the-terminal
import sys
old_stdout = sys.stdout
sys.stdout = open('trainingLog.txt', 'w')

In [ ]:
%cd models/research
%run object_detection/legacy/train.py --logtostderr --train_dir=../../trainingOutput/ --pipeline_config_path=../../FromScratch/models/model/ssd_mobilenet_v1_coco.config 
%cd ../..

W0404 12:31:59.532348 46956517787008 deprecation.py:323] From /home-1/cpaolic1@jhu.edu/.conda/envs/LPEnvironment/lib/python3.7/site-packages/absl/app.py:250: main (from __main__) is deprecated and will be removed in a future version.
Instructions for updating:
Use object_detection/model_main.py.
W0404 12:31:59.533112 46956517787008 module_wrapper.py:139] From /home-net/home-1/cpaolic1@jhu.edu/code/AutomaticLP/models/research/object_detection/legacy/train.py:91: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

W0404 12:31:59.534191 46956517787008 module_wrapper.py:139] From /home-1/cpaolic1@jhu.edu/code/AutomaticLP/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0404 12:31:59.538302 46956517787008 module_wrapper.py:139] From /home-net/home-1/cpaolic1@jhu.edu/code/AutomaticLP/models/research/object_detection/legacy/train.py:96: The name tf.gfile.Copy is deprecated.

In [ ]:
# Put the cell output back to normal
sys.stdout = old_stdout

In [ ]:
%matplotlib inline
%run viewTraining.py

## Evaluation

Both Evaluation and Training outputs can be looked at in Tensorboard for extra details

**TODO: update checkpoint number based on number of steps trained to** 

In [ ]:
%cd models/research
!python object_detection/export_inference_graph.py --input_type image_tensor --pipeline_config_path ../../FromScratch/models/model/ssd_mobilenet_v1_coco.config --trained_checkpoint_prefix ../../trainingOutput/model.ckpt-10 --output_directory ../../inference_graph
%cd ../..

In [ ]:
%matplotlib inline
%run customEvaluation.py

In [ ]:
%cd models/research
!python object_detection/legacy/eval.py --logtostderr --pipeline_config_path=../../FromScratch/models/model/ssd_mobilenet_v1_coco.config --checkpoint_dir=../../trainingOutput --eval_dir=../../evalOutput
%cd ../..